In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re, string


from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.stats import uniform
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import OneHotEncoder

def accuracy(y, t):
    return np.mean(y == t)

"""def accuracy(y, t):
    return np.mean(y[y>0] == t[y>0])"""

'def accuracy(y, t):\n    return np.mean(y[y>0] == t[y>0])'

In [34]:
if __name__ == "__main__":
    df_n = pd.read_csv("UofT_nodes.csv")
    df_n.dropna(inplace=True)
    df_occupation = pd.read_csv("UofT_occupation_risk.csv")
    df_n["birth_year"]=df_n["BIRTH_DT"].str.split("-").str[0]
    df_n["cust_add_year"]=df_n["CUST_ADD_DT"].str.split("-").str[0]
    df_n["ocptn_risk"]=df_n["OCPTN_NM"].map(df_occupation.set_index("code")["occupation_risk"])

In [35]:
    cols=["CASH_SUM_IN","CASH_SUM_OUT","WIRES_SUM_IN","WIRES_SUM_OUT"]
    df_n[cols]=df_n[cols].astype(int)
    df_n.drop(["BIRTH_DT","CUST_ADD_DT","CUSTOMER_ID","NAME","OCPTN_NM"],axis=1,inplace=True)
    #df_n=pd.get_dummies(df_n,prefix={"GENDER":"is","COUNTRY_RISK_INCOME":"inc_risk","COUNTRY_RISK_RESIDENCY":"res_risk"},columns=["GENDER","COUNTRY_RISK_INCOME","COUNTRY_RISK_RESIDENCY"],drop_first=True)
    df_n.info()
    #df_n.drop(["GENDER","is_Female","COUNTRY_RISK_INCOME","COUNTRY_RISK_RESIDENCY"],axis=1,inplace=True)
    pd.unique(df_n["GENDER"])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 988820 entries, 0 to 999999
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   RES_CNTRY_CA            988820 non-null  int64  
 1   CNTRY_OF_INCOME_CA      988820 non-null  int64  
 2   PEP_FL                  988820 non-null  float64
 3   CASH_SUM_IN             988820 non-null  int32  
 4   CASH_CNT_IN             988820 non-null  float64
 5   CASH_SUM_OUT            988820 non-null  int32  
 6   CASH_CNT_OUT            988820 non-null  float64
 7   WIRES_SUM_IN            988820 non-null  int32  
 8   WIRES_CNT_IN            988820 non-null  float64
 9   WIRES_SUM_OUT           988820 non-null  int32  
 10  WIRES_CNT_OUT           988820 non-null  float64
 11  COUNTRY_RISK_INCOME     988820 non-null  object 
 12  COUNTRY_RISK_RESIDENCY  988820 non-null  object 
 13  RISK                    988820 non-null  object 
 14  GENDER              

array(['Female', 'Male'], dtype=object)

In [36]:
X_cat=df_n[["GENDER","COUNTRY_RISK_INCOME","COUNTRY_RISK_RESIDENCY"]].astype('str').to_numpy()
X=df_n.iloc[:,np.r_[0:10,15:17]].to_numpy()
t=df_n.iloc[:,13].astype('str').to_numpy()
enc=OneHotEncoder(handle_unknown='ignore',drop=["Female","Low","Low"],sparse_output=False)
X_cat_enc=enc.fit_transform(X_cat)
print(X_cat[:10,:])
print(X_cat_enc[:10,:])
X=np.concatenate((X,X_cat_enc),axis=1)


[['Female' 'Low' 'Low']
 ['Male' 'Low' 'Low']
 ['Male' 'Low' 'Low']
 ['Female' 'Low' 'Low']
 ['Female' 'Low' 'Low']
 ['Female' 'Low' 'Low']
 ['Female' 'Low' 'Low']
 ['Male' 'Low' 'Low']
 ['Female' 'Low' 'Low']
 ['Female' 'Low' 'Low']]
[[0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [37]:
X_train, X_valid, t_train, t_valid = train_test_split(X,t,test_size=0.3,random_state=42)

In [38]:
for depth in [5,10,15,20,25]:
    
    clf=RandomForestClassifier(n_estimators=20,max_depth=depth, random_state=30)
    clf.fit(X_train,t_train)
    print("at depth ",depth," training acc: ",accuracy(clf.predict(X_train),t_train))
    print("at depth ",depth," validation acc: ",accuracy(clf.predict(X_valid),t_valid))

at depth  5  training acc:  0.9464599941633173
at depth  5  validation acc:  0.9460501742818039
at depth  10  training acc:  0.9501108102875866
at depth  10  validation acc:  0.9491852241392097
at depth  15  training acc:  0.963373371435506
at depth  15  validation acc:  0.953378774701159
at depth  20  training acc:  0.9861624389243168
at depth  20  validation acc:  0.9542687243381
at depth  25  training acc:  0.9976884425014519
at depth  25  validation acc:  0.9542619822953958


In [ ]:
accuracy(clf.predict(X))


In [ ]:
rf=RandomForestClassifier()